In [37]:
###########################     Introduction aux plus proches voisins # k nearest neighbors KNN       #####################

# base de travail dataset de airbNB_paris
# objectif : trouver les plus proches annonces d'appartement à Paris pour déterminer le prix de l'annonce 
# fixer le prix du marché de la location à mettre en ligne selon l'optimisation des données par l'algorithme KNN


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
# lire le jeu de données avec pandas.read_csv dans une variable paris_listings

paris_listings = pd.read_csv("paris_airbnb.csv")
paris_listings.iloc[0]

host_response_rate                 100%
host_acceptance_rate                NaN
host_listings_count                 1.0
latitude                       48.83349
longitude                       2.31852
city                              Paris
zipcode                           75014
state                     Île-de-France
accommodates                          2
room_type               Entire home/apt
bedrooms                            0.0
bathrooms                           1.0
beds                                0.0
price                            $75.00
cleaning_fee                     $50.00
security_deposit                  $0.00
minimum_nights                        3
maximum_nights                       15
number_of_reviews                     8
Name: 0, dtype: object

In [40]:
# stratégie trouver des logements similaires, calculer le prix moyens de ces logements en définir le prix du futur logement en location

# distance euclidienne pour métrique de similarité 

# distance euclidienne = valeur absolue de (q1 - p1)
# les q1 jusqu'à qn cela représente les valeurs de la caratéristiques pour une observation
# et les valeurs p1 jusqu'à pn cela représente les valeurs de la caratéristiques pour l'autre observation
# observation est un logement et une caractéristique peut être une colonne de notre dataframe comme 'beds' ou 'bathrooms'
# numpy.abs(q1-p1)


In [41]:
# numpy.abs(q1-q2)

In [42]:
# variable qui contient les valeurs de la colonne 'accommodates' le logement peut accueillir 3 personnes
our_acc_value = 3
# variable qui contient les valeurs de la colonne 'accommodates' pour le premier logement du dataset index 0
first_living_space = paris_listings.iloc[0]['accommodates']
# pour calculer la dist euclidienne avec la méthode de numpy 
# --> la valeur absolue de ( la première valeur de 'accomodates' dans le data set - le nb de personnes que peut accueillir le logement à mettre en locaton ) 
first_distance = np.abs(first_living_space - our_acc_value)
first_distance

1

In [43]:
# comprarer la distance entre un logement à 8 capacités d'accueil aux autres logement du dataframe col 'accomodates'
# coder les résultats par odre croissant

our_acc_value = 8
livings_spaces = paris_listings['accommodates']
distances = np.abs(livings_spaces - our_acc_value) 

list_distances = pd.Series(distances)

list_distances.head(13)

0     6
1     6
2     4
3     5
4     6
5     6
6     4
7     5
8     4
9     6
10    2
11    6
12    6
Name: accommodates, dtype: int64

In [44]:
# comprarer la distance entre un logement à 8 capacités d'accueil aux autres logement du dataframe col 'accomodates'
# coder les résultats par odre croissant

our_acc_value = 3
livings_spaces = paris_listings['accommodates']
distances = np.abs(livings_spaces - our_acc_value) 

list_distances = pd.Series(distances)
# Afficher le décompte des valeurs de distance
distance_counts = list_distances.value_counts()
print(distance_counts)

accommodates
1     5665
0      878
2      726
3      508
4       88
5       82
6       18
7       14
9        8
11       4
8        4
13       3
10       2
Name: count, dtype: int64


In [45]:
''' Calculer la distance pour toutes les observations

* Calculer la distance entre chaque valeur de la colonne 'accommodates' de paris_listings et la valeur 3 (qui est le nombre de personnes que peut accueillir notre logement):
 - Utiliser la méthode **apply** pour calculer la valeur absolue entre chaque valeur de 'accommodates' et 3 et retourner un nouvel objet Series contenant les valeurs de distance.
* Assigner les valeurs de distance à la colonne 'distance' du dataframe.
* Utiliser la méthode Series value_counts() et afficher le décompte de chaque valeur unique de la colonne 'distance'.'''

########## Correction instructeur   ########

# écrire une fonction lambda sur la variable x, pour éviter d'écrire une fonction séparée 
# cette fonction va retourner un objet series où la valeur vaut 2 de plus que la valeur d'orignine de la col 'accomodates'
# exemple : df['accomodates'].apply(lambda x: x + 2)

new_listing = 3
paris_listings['distance'] = paris_listings['accommodates'].apply(lambda x: np.abs(x - new_listing))
# ft lambda --> valeur absolue de chaque logement x (ici col 'accomodates' - notre logement valeur de 3 ) une nouvelle col est crée
print(paris_listings['distance'].value_counts())

distance
1     5665
0      878
2      726
3      508
4       88
5       82
6       18
7       14
9        8
11       4
8        4
13       3
10       2
Name: count, dtype: int64


In [46]:
## Randomiser et trier

paris_listings[paris_listings['distance'] == 0]['accommodates']
# en cherchant les logements similaires à un accueil de 3 personnes, chercher un échantillon aléatoire diff de ce résultat

3       3
7       3
13      3
14      3
18      3
       ..
7907    3
7938    3
7948    3
7958    3
7989    3
Name: accommodates, Length: 878, dtype: int64

In [47]:
paris_listings.head(2)

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,distance
0,100%,NaN,1.0,48.83349,2.31852,Paris,75014,Île-de-France,2,Entire home/apt,0.0,1.0,0.0,$75.00,$50.00,$0.00,3,15,8,1
1,100%,NaN,1.0,48.85100,2.35869,Paris,75004,Ile-de-France,2,Entire home/apt,0.0,1.0,1.0,$115.00,$36.00,$0.00,1,90,173,1


In [48]:
''' * On commence par se placer sur le même seed: np.random.seed(1)
* Randomiser l'ordre des lignes de paris_listings:
 - Utiliser la fonction np.random.permutation() pour retourner un tableau NumPy de valeurs d'index mélangés.
 - Utiliser la méthode sur Dataframe loc[] pour retourner un nouveau DataFrame contenant l'ordre mélangé.
 - Assigner le nouveau Dataframe de nouveau à paris_listings.
* Après avoir randomisé, trier paris_listings sur la colonne 'distance' avec la méthode sort_values().
* Afficher les 10 premières valeurs de la colonne 'price'.'''

np.random.seed(1)
paris_listings = paris_listings.loc[np.random.permutation(len(paris_listings))]
paris_listings = paris_listings.sort_values('distance')
paris_listings.iloc[0:10]['price']

1956     $80.00
3735     $67.00
6944     $36.00
2094    $120.00
2968     $60.00
845     $200.00
6083     $95.00
5970     $95.00
2992     $76.00
3179    $100.00
Name: price, dtype: object